In [1]:
%matplotlib inline
import os
import loompy
import logging
import pandas as pd
import numpy as np
import cytograph as cg
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import *
from sklearn.metrics.pairwise import cosine_distances
import numpy_groupies as npg
import matplotlib.pyplot as plt

In [18]:
with loompy.connect("/Users/stelin/build_20171205/L5_All.agg.loom") as ds:
    gene = "Meis2"
    ct = ds.ca.ClusterName[ds.ca.TaxonomyRank4 == "Telencephalon inhibitory interneurons"]
    mu = ds[ds.ra.Gene == gene, :][:,ds.ca.TaxonomyRank4 == "Telencephalon inhibitory interneurons"][0]
    tr = ds["trinaries"][ds.ra.Gene == gene, :][:,ds.ca.TaxonomyRank4 == "Telencephalon inhibitory interneurons"][0]
    print(gene)
    ordering = np.argsort(ct)
    ct = ct[ordering]
    mu = mu[ordering]
    tr = tr[ordering]
    for ix in range(len(ct)):
        print(ct[ix], mu[ix], tr[ix]>0.95)

Meis2
DEINH1 1.57522123894 True
DEINH2 2.1497005988 True
TEINH10 0.0 False
TEINH11 0.00454545454545 False
TEINH12 0.0 False
TEINH13 0.0306748466258 False
TEINH14 0.021327014218 False
TEINH15 0.0159027128157 False
TEINH16 0.00155279503106 False
TEINH17 0.04 False
TEINH18 0.0109689213894 False
TEINH19 0.00576923076923 False
TEINH20 1.69064748201 True
TEINH21 0.0753246753247 False
TEINH4 0.0108527131783 False
TEINH5 0.00445434298441 False
TEINH6 0.0 False
TEINH7 0.0 False
TEINH8 0.0 False
TEINH9 0.0 False


In [31]:
with loompy.connect("/Users/stelin/build_20171205/L5_All.agg.loom") as ds:
    enrichment = ds["enrichment"][:,:]
    genes = []
    for ix in range(ds.shape[1]):
        genes += list(ds.ra.Gene[np.argsort(-enrichment[:, ix])[:5]])
    print(np.unique(genes))

['1500015O10Rik' '1700012B09Rik' '1700042O10Rik' '2210011C24Rik'
 '2410004P03Rik' '2810417H13Rik' '2810459M11Rik' '3110035E14Rik'
 '4921539H07Rik' '4930426D05Rik' '4933407L21Rik' '5330429C05Rik'
 'A230065H16Rik' 'A2m' 'A830009L08Rik' 'A830036E02Rik' 'AW551984' 'Abi3bp'
 'Acbd7' 'Acta2' 'Ada' 'Adarb2' 'Adcyap1' 'Adgrg2' 'Adgrg6' 'Adm' 'Adora2a'
 'Adra1b' 'Afp' 'Agrp' 'Agt' 'Ahnak2' 'Ankrd34b' 'Anln' 'Apln' 'Apold1'
 'Aqp1' 'Aqp4' 'Arap2' 'Areg' 'Arg1' 'Arhgap36' 'Arhgap6' 'Arsj' 'Asb4'
 'Asic4' 'Ass1' 'Atf3' 'Atp2a3' 'Avp' 'B3gat2' 'BC039966' 'Baiap2l1'
 'Baiap3' 'Barhl1' 'Bcl2a1b' 'Bglap' 'Bhlhe22' 'Birc2' 'Birc5' 'Bmp3'
 'Bmp4' 'C1ql1' 'C1ql2' 'C1ql3' 'C4b' 'Cabp7' 'Cacna2d2' 'Calb1' 'Calb2'
 'Calca' 'Calcb' 'Car8' 'Cartpt' 'Cbln1' 'Cbln2' 'Cbln3' 'Cbln4' 'Ccdc153'
 'Cckar' 'Ccl24' 'Ccl3' 'Ccl4' 'Ccl7' 'Ccp110' 'Ccrl2' 'Cd93' 'Cdc20'
 'Cdhr1' 'Cdk1' 'Cdkl4' 'Cdkn1c' 'Cemip' 'Cenpf' 'Cfap126' 'Chat' 'Chodl'
 'Chrm2' 'Chrna2' 'Chrna6' 'Chrnb3' 'Cks2' 'Clca3a1' 'Cldn10' 'Cldn19'
 'Cldn5'

In [32]:
np.unique(genes).shape

(667,)

## TODO

1. Make table of expression for Kenneth's genes
2. Make function to measure specificity of a gene set
3. Reduce gene set gradually and plot specificity
4. Calculate number of genes and total expression per cluster

Same analysis on interneurons using the 99 genes